## Toronto Crime Analysis


### This notebook reads crime data from the Toronto Police API in chunks (due to API limit), combines and cleans it, and converts it to a csv, to be imported by other notebooks in this project

In [1]:
import requests
import json
import pandas as pd
from pprint import pprint
from pandas.io.json import json_normalize
from IPython.display import clear_output

In [58]:
# Toronto police major crime indicator data:
mci_data = requests.get('https://services.arcgis.com/S9th0jAJ7bqgIRjw/arcgis/rest/services/MCI_2014_2017/FeatureServer/0/query?where=occurrenceyear%20%3E%3D%202014%20AND%20occurrenceyear%20%3C%3D%202014%20AND%20UPPER(occurrencemonth)%20like%20%27%25JANUARY%25%27&outFields=*&outSR=4326&f=json').json()

In [59]:
mci_data.keys()

dict_keys(['objectIdFieldName', 'uniqueIdField', 'globalIdFieldName', 'geometryType', 'spatialReference', 'fields', 'features', 'exceededTransferLimit'])

In [60]:
# look into each key:
mci_data['objectIdFieldName']

'FID'

In [62]:
# only 'features' key has any actual data
mci_data['features']

[{'attributes': {'Index_': 701,
   'event_unique_id': 'GO-20141410316',
   'occurrencedate': 1390626000000,
   'reporteddate': 1390626000000,
   'premisetype': 'Apartment',
   'ucr_code': 1430,
   'ucr_ext': 100,
   'offence': 'Assault',
   'reportedyear': 2014,
   'reportedmonth': 'January',
   'reportedday': 25,
   'reporteddayofyear': 25,
   'reporteddayofweek': 'Saturday  ',
   'reportedhour': 20,
   'occurrenceyear': 2014,
   'occurrencemonth': 'January',
   'occurrenceday': 25,
   'occurrencedayofyear': 25,
   'occurrencedayofweek': 'Saturday  ',
   'occurrencehour': 20,
   'MCI': 'Assault',
   'Division': 'D43',
   'Hood_ID': 127,
   'Neighbourhood': 'Bendale (127)',
   'Lat': 43.7544174,
   'Long': -79.2509308,
   'FID': 103},
  'geometry': {'x': -79.2509308, 'y': 43.7544174}},
 {'attributes': {'Index_': 702,
   'event_unique_id': 'GO-20141411089',
   'occurrencedate': 1390626000000,
   'reporteddate': 1390626000000,
   'premisetype': 'Apartment',
   'ucr_code': 1430,
   'ucr_e

In [63]:
len(mci_data['features'])

2000

In [64]:
# 2000 but they should be ~131,000
# this api limits to 2000 records
# we'll loop per chunks of time to get all records

In [122]:
years = range(2014,2018)
daysofyear = range(1,365,10)

In [123]:
# Toronto police major crime indicator data:
mci_data=[]
for year in years:
    for day in daysofyear:
        clear_output(wait=True)
        
        response = requests.get('https://services.arcgis.com/S9th0jAJ7bqgIRjw/arcgis/rest/services/MCI_2014_2017/FeatureServer/0/query?where=occurrenceyear%20%3E%3D%20' + str(year) + '%20AND%20occurrenceyear%20%3C%3D%20' + str(year) + '%20AND%20occurrencedayofyear%20%3E%3D%20' + str(day) + '%20AND%20occurrencedayofyear%20%3C%3D%20' + str(day+10) + '&outFields=Index_,event_unique_id,occurrencedate,premisetype,offence,occurrenceyear,occurrencemonth,occurrenceday,occurrencedayofyear,occurrencedayofweek,occurrencehour,Hood_ID,Neighbourhood,MCI&outSR=4326&f=json').json()
        for record in response['features']:
            mci_data.append(record)
        
        # was taking a long time, found this way to show progress
        if (year == 2017 and day > 350):
            print( "Done")
        else:
            print( "fetching data for days " + str(day) + " to " + str(day+10) + " of year " + str(year) )
        

Done


In [124]:
len(mci_data)
# larger than expected, must have duplicates

142983

In [125]:
crimes_df_raw = json_normalize(mci_data)
crimes_df_raw.head()

,attributes.Hood_ID,attributes.Index_,attributes.MCI,attributes.Neighbourhood,attributes.event_unique_id,attributes.occurrencedate,attributes.occurrenceday,attributes.occurrencedayofweek,attributes.occurrencedayofyear,attributes.occurrencehour,attributes.occurrencemonth,attributes.occurrenceyear,attributes.offence,attributes.premisetype,geometry.x,geometry.y
0,84,976,Assault,Little Portugal (84),GO-2015942420,1388552400000,1,Wednesday,1,0,January,2014,Assault,Apartment,-79.426590,43.651516
1,25,977,Assault,Glenfield-Jane Heights (25),GO-20162295518,1388552400000,1,Wednesday,1,0,January,2014,Assault,Apartment,-79.518127,43.754486
2,71,978,Assault,Cabbagetown-South St.James Town (71),GO-20142458487,1388552400000,1,Wednesday,1,0,January,2014,Assault,Apartment,-79.370407,43.670261
3,71,979,Assault,Cabbagetown-South St.James Town (71),GO-20142458487,1388552400000,1,Wednesday,1,0,January,2014,Assault,Apartment,-79.370407,43.670261
4,58,980,Theft Over,Old East York (58),GO-20141573896,1388552400000,1,Wednesday,1,0,January,2014,Theft Over,House,-79.344810,43.693340


In [126]:
# remove extra columns
crimes_df = crimes_df_raw.drop(['geometry.x', 'geometry.y'], axis=1).drop_duplicates()

In [127]:
# remove 'attributes' from headers
crimes_df.columns = crimes_df.columns.str.strip().str.replace('attributes.','')
crimes_df.head()

,Hood_ID,Index_,MCI,Neighbourhood,event_unique_id,occurrencedate,occurrenceday,occurrencedayofweek,occurrencedayofyear,occurrencehour,occurrencemonth,occurrenceyear,offence,premisetype
0,84,976,Assault,Little Portugal (84),GO-2015942420,1388552400000,1,Wednesday,1,0,January,2014,Assault,Apartment
1,25,977,Assault,Glenfield-Jane Heights (25),GO-20162295518,1388552400000,1,Wednesday,1,0,January,2014,Assault,Apartment
2,71,978,Assault,Cabbagetown-South St.James Town (71),GO-20142458487,1388552400000,1,Wednesday,1,0,January,2014,Assault,Apartment
3,71,979,Assault,Cabbagetown-South St.James Town (71),GO-20142458487,1388552400000,1,Wednesday,1,0,January,2014,Assault,Apartment
4,58,980,Theft Over,Old East York (58),GO-20141573896,1388552400000,1,Wednesday,1,0,January,2014,Theft Over,House


In [128]:
len(crimes_df)

130167

In [129]:
# output to csv
crimes_df.to_csv('crimes_csv.csv')